# GS541 phylogenetics homework 1

## Poisson distribution and estimation via maximum likelihood

At the end of this segment, I hope you understand

* why we use the Poisson distribution to model mutation processes
* how model parameter estimation via maximum likelihood "works"

In [ ]:
from scipy.stats import poisson

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

In [ ]:
def sample_event(probability):
    """Return True with probability `probability`, otherwise return False."""
    return random.random() < probability

Demonstrate that your code is working properly by summing together a large number of outputs from this function and seeing that it's giving (approximately) the expected number of Trues.

In [ ]:
sum(sample_event(0.9) for i in range(1000))

In [ ]:
def count_mutations(generation_count, mutation_probability):
    """Simulate the number of mutations after `generation_count` generations, 
    each of which mutates with probability `mutation_probability`."""
    return sum(sample_event(mutation_probability) for i in range(generation_count))

In [ ]:
count_mutations(1000, 0.01)

In [ ]:
mutation_counts = pd.Series([count_mutations(700, 0.01) for i in range(10000)])

In [ ]:
mutation_counts.plot.hist(bins=max(mutation_counts))

In [ ]:
ax = mutation_counts.plot.density(bw_method=0.25)

x = range(12) 
pois_df = pd.DataFrame({"x":x, "pdf": poisson.pmf(x, 7.)})
pois_df.plot(x="x", y="pdf", ax=ax)

Make a plot of the Poisson distribution with various means.
Fit one "by eye".

Mutations versus substitutions versus observed mutations.

1. start with a string
2. for every generation, for every site, flip the state
3. note that we aren't getting the right number
3. estimate the number of actual substitutions

In [ ]:
sequence = "0"*1000


1. start with a string
2. for every generation, for every site, sample the number of mutations that will happen, and for each one sample the base that will be sampled
3. estimate the number 